# 필기 한자 인식모델 구현 및 히라가나 변환
# 모델 use

<div style="text-align: right"> 산업경영공학과 </div>
<div style="text-align: right"> 2018100922 </div>
<div style="text-align: right"> 이승건 </div>

참고) 경로는 사용자 환경에 맞게 설정하여 사용하도록 함.

## Colab에서 드라이브 연동하기

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

## 라이브러리

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

## 데이터 불러오기

In [ ]:
y_train = np.load('data/y_train.npy')
y_train_char = np.load('data/y_train_char.npy')

## jpg to array 함수

In [ ]:
def jpg_image_to_array(image_path, size1, size2):
  image = Image.open(image_path)
  image = image.resize((size1, size2))
  im_arr = np.fromstring(image.tobytes(), dtype = np.uint8)
  print(im_arr.shape)
  im_arr = im_arr.reshape((image.size[1], image.size[0], 3))
  print(im_arr.shape)
  im_arr = im_arr*(-1) + 255
  return im_arr

def image_array_to_1channel(arr):
  arr_avg = (arr[:,:,0] + arr[:,:,1] + arr[:,:,2])/3
  return arr_avg

## 모델 불러오기

In [ ]:
from tensorflow.python.keras.models import load_model

model = load_model("model/cnn_japanese_96_19.h5")
model.summary()

## japanese_use_firstname 내 데이터 예측하기

In [ ]:
import os
dir = "japanese_use_firstname"
subdir_names = os.listdir(dir)
X_test = []
y_test = []
ignore_data = []
predict_data = []
#subdir_name = 폴더 단위
for subdir_name in subdir_names:
  count = 0
  subdir = dir +"/" + subdir_name
  predict_data_add = []
  #file names 순서 오류 해결
  file_names = os.listdir(subdir)
  file_names_sort = []
  for k in subdir_name:
    if len(file_names) != 0:
      index_sort = [i for i in range(len(file_names)) if k in file_names[i]]
      file_names_sort.append(file_names[index_sort[0]])
  #file_name = 파일단위
  for file_name in file_names_sort:
    if len(np.where(y_train_char ==  subdir_name[count])[0]) == 0:
      ignore_data.append(subdir_name[count])
    else:
      path = subdir +"/" + file_name
      print('filename:', file_name)
      arr = jpg_image_to_array(path, 128, 127)
      arr_1c = image_array_to_1channel(arr)
      print(arr_1c.shape)
      if len(X_test) == 0:
        X_test = [arr_1c]
        y_test = int(y_train[np.where(y_train_char ==  subdir_name[count])[0][0]])
        predict_data_add.append(subdir_name[count])
        print(subdir_name[count])
      else:
        X_test = np.concatenate((X_test, [arr_1c]))
        y_test = np.append(y_test, int(y_train[np.where(y_train_char ==  subdir_name[count])[0][0]]))
        predict_data_add.append(subdir_name[count])
        print(subdir_name[count])
    count += 1
  if predict_data_add != []:
    predict_data.append(predict_data_add)

In [ ]:
first_image = X_test[5]
first_image = np.array(first_image, dtype = 'float')
#pixels = first_image.reshape((28,28))
plt.imshow(first_image, cmap = 'gray')
plt.show

In [ ]:
print(X_test.shape)
X_test = np.expand_dims(X_test, axis = -1)
print(X_test.shape)

In [ ]:
#실제 값
y_test

In [ ]:
#예측 값
y_pred = model.predict_classes(X_test)
y_pred

In [ ]:
# 학습 모델에 없는 값
ignore_data

In [ ]:
# 실제 값
predict_data

In [ ]:
# 예측 값
y_pred_add = []
for i in range(len(y_pred)):
  y_pred_add = np.append(y_pred_add, [y_train_char[np.where(y_train == y_pred[i])[0][0]][0]])
y_pred_real = []
y_pred_reshape = []
len_y_pred = 0
for i in predict_data:
  y_pred_reshape = y_pred_add[len_y_pred:len_y_pred + len(i)]
  len_y_pred += len(y_pred_reshape)
  y_pred_real.append(y_pred_reshape)

In [ ]:
#예측 값
y_pred_real

### 테스트 정확도 검증

In [ ]:
from tensorflow.keras import utils
y_binary = utils.to_categorical(y_test, 881)
print(y_binary)

score = model.evaluate(X_test, y_binary)  
print('test_loss: ', score[0])
print('test_acc: ', score[1])

## 히라가나 변환

### 일본 성씨 데이터 crawling

In [ ]:
df = pd.read_csv('japanese-family-names-master/myoji-yurai-readings.csv', names = ['name', 'reading', 'num', 'difficult'])

In [ ]:
#pd.set_option('display.max_rows', None)

In [ ]:
df

In [ ]:
predict_data

In [ ]:
y_pred_real

### 히라가나 변환 결과

In [ ]:
for i in y_pred_real:
  string_join = ''.join(i)
  hiragana = df['reading'].values[df['name'] == string_join]
  if hiragana.size == 0:
    print(string_join, '=', "학습 결과가 올바르지 않습니다.")
  else:
    if '|' in hiragana[0]:
      index = hiragana[0].find('|')
      print(string_join, '=', hiragana[0][:index], '또는', hiragana[0][index+1:], '입니다.')
    else:
      print(string_join, '=', hiragana[0], '입니다.')